# Model Predictive Control Rocket Soft-Landing Problem

This notebook is a walkthough of the ALTRO solver for the MPC Rocket Soft-Landing Problem (i.e. the general control challenge of landing a rocket at rest at a designated landing site). In the simplest case, we treat the rocket as a sphere landing on a flat surface with no atmosphere. This approximation is reasonable when the rocket is near the ground and entering the final part of the descent.

**This files focuses on the MPC problem and comparing the result to other solvers. For the simple (non-MPC) application and set-up explanation, please visit the <code>RocketLandingSOCP-SetupExplained.ipynb</code> file. Thanks!**

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

## MPC Problem Outline

For the MPC application, we must fulfill two critical steps:
- [x] Solve the optimal trajectory and apply the required controls (usually a short horizon)
- [x] Use the prior trajectory to warm-start the next solve

In this Jupyter notebook, we will:
1. Set up the rocket soft-landing problem
    - Model Dynamics
    - Discretization
    - Specify the Disturbance (e.g. wind) separate from model dynamics
    - Initial and Final States
2. Solve the initial trajectory optimization problem (cold-start)
    - Solve the full trajectory
3. Solve the realtime trajectory optimization problem (warm-start, shorter horizon)
    - Solve the shorter trajectory
    - Execute the control (and propagate forward in time)
    - Warm-Start the next iteration
    - Solve the next shorter horizon trajectory optimization problem
    - Repeat above steps until arrival at the landing location

# Step 1: Set-Up the Rocket Soft-Landing Problem

## Load Required Packages

The <code>RobotDynamics</code> package enables us to define the rocket model and dynamics. The <code>TrajectoryOptimization</code> and <code>Altro</code> set-up and solve the trajectory optimization problem. <code>StaticArrays</code> and <code>LinearAlgebra</code> are required to set-up the appropriate data types.

Next, <code>Plots, MeshCat, TrajOptPlots, FileIO,</code> and <code>MeshIO</code> packages are used to visualize the results. Note that the plotting sections will take significantly longer to run than the optimization. 

Lastly, <code>DifferentialEquations</code> integrates the dynamics forward as a way to show that the dynamics constraints are satisfied.

Loading all the packages, especially for plotting and the differential equations integrator can be the longest part of the whole notebook.

In [ ]:
using RobotDynamics
import RobotDynamics: dynamics
using Rotations
using Altro
using TrajectoryOptimization
using StaticArrays, LinearAlgebra
println("Trajectory Optimization Packages Ready")

# using Plots
# pyplot()
# println("Base Plotting Packages Ready")

# using MeshCat, TrajOptPlots
# using FileIO, MeshIO
# println("3D Plotting Packages Ready")

import DifferentialEquations.solve, DifferentialEquations.ODEProblem, DifferentialEquations.Tsit5
using BenchmarkTools
println("Differential Equations Ready")

Add second-order cone constraints

In [ ]:
include("general_socp.jl")
println("Added more general SOCP Constraint")

## Detail the Rocket Dynamics

For the simplified rocket problem, we choose to ignore the attitude dynamics. Applying newton's second law or by forming a Lagrangian, we arrive at the following sparse linear system

$$\frac{dx}{dt} = Ax + Bu + g$$

Where $A = \begin{bmatrix}0 & I\\ 0 & 0 \end{bmatrix}$, $B = \begin{bmatrix}0 \\ \frac{1}{m} I\end{bmatrix}$, and $G = \begin{bmatrix} 0 \\ -g \end{bmatrix}$. For the 3D case, these are $(6 \times 6)$, $(6 \times 3)$, and $(6 \times 1)$ matricies, respectively. 

In [ ]:
const mass = 10.0
const grav = SA[0.0; 0.0; -9.81]

model = LinearModel([zeros(3,3) I; zeros(3, 6)], [zeros(3,3); (1/mass) * I], vec([zeros(3, 1); grav]))

struct Rocket
    m
    g
    model
end

## Instantiate the Model and Define the Discretization

Feel free to choose <code>N</code> and <code>tf</code> based on your needs.

In [ ]:
rocket = Rocket(mass, grav, model)
n, m = size(rocket.model)

# Trajectory Discretization
N = 251
tf = 10.
dt = tf/(N-1)

rocket # (Simply for information) Display the struct as the cell output 

## Detail the Full Dynamics (With Distrubance)

In Matlab, we would use ODE 45, but in the <code>DifferentialEquations.jl</code> package we can either (1) just let the <code>solve</code> method figure it out or (2) specify a solver such as <code>Tsit5</code>, which should handle any ODE that a fourth-order Runge Kutta Method could handle.

In [ ]:
"Disturbance"
function wind(y, p, t)
    return [zeros(3); 0; 0.1; 0]
end

"Full Dynamics Model"
function full_model(r::Rocket, x0, dt::Float64, controlsArr, disturbance; num_timesteps::Int64 = 3, t0::Float64 = 0.0)
    # Zero-Order Hold
    get_controls(t) = controlsArr[min(Int(floor(t/dt)) + 1, size(controlsArr,1))]
    # Full Dynamics
    f(y,p,t) = dynamics(r.model, y, get_controls(t)) + disturbance(y, p, t)

    tspan = (t0, t0 + dt * num_timesteps)
    controls_p0 = get_controls(t0)

    # Use the DifferentialEquations.jl Package
    ode_prob = ODEProblem(f, x0, tspan, controls_p0)
    ode_sol = solve(ode_prob, Tsit5(), reltol=1e-10, abstol=1e-10);
    
    return ode_sol
end

## Specify the Initial and Final Conditions

Choose the initial and final points. Note that these must be full state vectors ($[x; y; z; v_x; v_y; v_z]$). The initial point is generally the rocket's current state. The final point is generally the rocket's landing stop. Note that the final velocity of the rocket should **always** be $[0.0; 0.0; 0.0]$ to have a **soft** landing.

In [ ]:
# Initial and Final Conditions
rocketStart = @SVector [4.0, 2.0, 20.0, -3.0, 2.0, -5.0] # Start at a 20 m altitude with an ~15 m/s plummet
rocketEnd = @SVector zeros(n);  # Swing pendulum up and end at rest

# Step 2: Solve Cold Start Trajectory

In [ ]:
# Helper function for Max Thrust Constraint
function getAlpha(theta, deg = true)
    if deg
        return tand(theta)
    end
    return tan(theta)
end

#=
Solve the trajectory optimization problem from current location to the final landing spot.

Returns the benchmark data and the optimized solve
=#
function ColdStartFull(r::Rocket, x0, xf, N, opts; 
                        Qk::Float64 = 1.0e-2, Qfk::Float64 = 100.0, Rk::Float64 = 1.0e-1, 
                        ground_level::Float64 = 0.0, theta_max_deg::Float64 = 7.0, perWeightMax::Float64 = 2.0,
                        includeGoal::Bool = true, verbose::Bool = true)
    n, m = size(r.model)
    
    # Set-UP the LQR Objective Function
    Q = Qk * Diagonal(@SVector ones(n))
    Qf = Qfk * Diagonal(@SVector ones(n))
    R = Rk * Diagonal(@SVector ones(m))
    obj = LQRObjective(Q,R,Qf,xf,N)
    if verbose
        println("Objective Set")
    end
    
    # Create Empty ConstraintList
    conSet = ConstraintList(n,m,N)
    
    if includeGoal
        # Goal Constraint that the rocket must reach the landing site.
        goal = GoalConstraint(xf)
        add_constraint!(conSet, goal, N)
        if verbose
            println("Goal Constraint Set")
        end
    end
    
    # Bound Constraint that the rocket doesn't crash into the ground
    # This constraint can be made more complicated for difficult terrain
    # This constraint can also be a glidescope constraint.
    bnd = BoundConstraint(n,m, x_min=[-Inf, -Inf, ground_level,
                                      -Inf, -Inf, -Inf])
    if verbose
        println("Ground Constraint Set")
    end
    
    # Norm Constrant that reflects the max thrust the rocket can provide
    u_bnd = r.m * norm(r.g[3]) * perWeightMax
    maxT = NormConstraint(n, m, u_bnd, TrajectoryOptimization.SecondOrderCone(), :control)
    add_constraint!(conSet, maxT, 1:N-1)
    if verbose
        println("Max Thrust Constraint Set")
    end

    # Generalized Norm Constraint that reflects the max thrust angle constraint
    # Based on the rocket gimbal
    maxTAalpha = getAlpha(theta_max_deg)
    ARocket = SizedMatrix{3,3}([1.0 0 0; 0 1.0 0; 0 0 0])
    cRocket = SVector{3}([0; 0; maxTAalpha])
    maxTA = NormConstraint2(n, m, ARocket, cRocket, TrajectoryOptimization.SecondOrderCone(), :control)
    add_constraint!(conSet, maxTA, 1:N-1)
    if verbose
        println("Max Thrust Angle Constraint Set")
    end
    
    # Package the objective and constraints into a "problem" type
    prob = Problem(model, obj, xf, tf, x0=x0, constraints=conSet)
    if verbose
        println("Problem Packaged")
    end
    
    # Set the initial controls to a hover
    u0 = r.g # controls that would nominally hover
    U0 = [u0 for k = 1:N-1] # vector of the small controls
    initial_controls!(prob, U0)
    rollout!(prob)
    if verbose
        println("Set Initial Controls")
    end
    
    altro = ALTROSolver(prob, opts)
    if verbose
        println("Set Solver and Options")
    end
    
    println("Prepping benchmark solve")
    set_options!(altro, show_summary=false)
    b = benchmark_solve!(altro)
    println("Benchmark solve")
    
    println("Prepping solve with summary")
    set_options!(altro, show_summary=true)
    solve!(altro);
    println("Solve Complete")
    
    return b, altro
end

In [ ]:
opts = SolverOptions(
        cost_tolerance_intermediate=1e-2,
        penalty_scaling=10.,
        penalty_initial=1.0,
        verbose = 1,
        projected_newton = false,
        constraint_tolerance = 1.0e-8
    )

b, altro_solve = ColdStartFull(rocket, rocketStart, rocketEnd, N, opts)

X_cold = states(altro_solve)
U_cold = controls(altro_solve);

In [ ]:
b

# Step 3: Solve Warm-Start and Model the Rocket Descent

In [ ]:
#=
Solve the trajectory optimization problem from current location to the final landing spot.

Returns the benchmark data and the optimized solve
=#
function MPC_SetUp(r::Rocket, x0, xf, horizon; 
                        Qk::Float64 = 1.0e-2, Rk::Float64 = 1.0e-1, 
                        ground_level::Float64 = 0.0, theta_max_deg::Float64 = 7.0, 
                        perWeightMax::Float64 = 2.0, verbose::Bool = true)
    n, m = size(r.model)
    
    # Set-UP the LQR Objective Function
    Q = Qk * Diagonal(@SVector ones(n))
    R = Rk * Diagonal(@SVector ones(m))
    
    obj = Objective(QuadraticCost(Q, R), horizon)
    if verbose
        println("Objective Set")
    end
    
    # Create Empty ConstraintList
    conSet = ConstraintList(n, m, horizon)
    
    # Bound Constraint that the rocket doesn't crash into the ground
    # This constraint can be made more complicated for difficult terrain
    # This constraint can also be a glidescope constraint.
    bnd = BoundConstraint(n,m, x_min=[-Inf, -Inf, ground_level,
                                      -Inf, -Inf, -Inf])
    if verbose
        println("Ground Constraint Set")
    end
    
    # Norm Constrant that reflects the max thrust the rocket can provide
    u_bnd = r.m * norm(r.g[3]) * perWeightMax
    maxT = NormConstraint(n, m, u_bnd, TrajectoryOptimization.SecondOrderCone(), :control)
    add_constraint!(conSet, maxT, 1:horizon-1)
    if verbose
        println("Max Thrust Constraint Set")
    end

    # Generalized Norm Constraint that reflects the max thrust angle constraint
    # Based on the rocket gimbal
    maxTAalpha = getAlpha(theta_max_deg)
    ARocket = SizedMatrix{3,3}([1.0 0 0; 0 1.0 0; 0 0 0])
    cRocket = SVector{3}([0; 0; maxTAalpha])
    maxTA = NormConstraint2(n, m, ARocket, cRocket, TrajectoryOptimization.SecondOrderCone(), :control)
    add_constraint!(conSet, maxTA, 1:horizon-1)
    if verbose
        println("Max Thrust Angle Constraint Set")
    end
    
    # Goal Constraint that the rocket must reach the landing site.
    goal = GoalConstraint(xf)
    add_constraint!(conSet, goal, horizon)
    if verbose
        println("Goal Constraint Set")
    end
    
    return obj, conSet
end

In [ ]:
o, c = MPC_SetUp(rocket, rocketStart, rocketEnd, 10, opts);

In [ ]:
function changeGoal!(c::ConstraintList, xf_new::SArray{Tuple{6},Float64,1,6})
    @assert typeof(c.constraints[end]) == GoalConstraint{6, Float64}
    c.constraints[end] = GoalConstraint(xf_new)
    
    return nothing
end


# Array{TrajectoryOptimization.AbstractConstraint,1}

function changeGoal!(p::Problem, xf_new::SArray{Tuple{6},Float64,1,6})
    return changeGoal!(p.constraints, xf_new)
end

function warmstart_problem(r::Rocket, obj::Objective, cset::ConstraintList, warm_states, warm_controls, tf)
    
    prob = Problem(r.model, obj, warm_states[end], tf, x0=warm_states[1], constraints=cset)
    
    initial_controls!(prob, warm_controls)
    initial_states!(prob, warm_states)
    rollout!(prob)
    
    return prob
end

function MPC_Solve(prob::Problem, opts)
    altro = ALTROSolver(prob, opts)
    
    set_options!(altro, show_summary=false)
    b = benchmark_solve!(altro)
    
    set_options!(altro, show_summary=true)
    solve!(altro);
    
    return b, altro
end

Execute the MPC forward until landed

In [ ]:
# Initial Conditions
x_curr = rocketStart
u_curr = [U_cold[1]]
t_curr = 0.0
num_tsteps = 1
hor = 20
tsteps = tf * hor / N


updated_model = full_model(rocket, x_curr, dt, u_curr, wind, num_timesteps = num_tsteps, t0 = t_curr)

t_curr = updated_model.t[end]
x_curr = updated_model.u[end]
display(x_curr - X_cold[2])

x_target = X_cold[hor]
changeGoal!(c, x_target)

X_warm = [[x_curr]; X_cold[3:hor + 1]]
U_warm = U_cold[2:hor+1]

p = warmstart_problem(rocket, o, c, X_warm, U_warm, tsteps)

b, altro_mpc = MPC_Solve(p, opts)
display(b)

In [ ]:
completion_threshold = 1

opts = SolverOptions(
    cost_tolerance_intermediate=1e-2,
    penalty_scaling=10.,
    penalty_initial=1.0
)

saved_states = []

while norm(x0 - xf) > completion_threshold
    x0, warm_start_controls, N, tf = MPC(model, ALTROSolver, x0, warm_start_controls, N, tf)
    
    println("\nCurrently at time $tf remaining and $(norm(x0 - xf)) away.")
    println("$N Steps at $tf s left. Hence, dt = $(tf / (N-1))")
    
    # If the timespan is too low (e.g. due to disturbances)
    tf = max(0.5, tf)
    
    push!(saved_states, x0)
end

In [ ]:
xs = [x[1] for x in saved_states]
zs = [x[3] for x in saved_states];

x_orig = [x[1] for x in X]
z_orig = [x[3] for x in X];

## Plot the trajectory!

In [ ]:
titleText = "Rocket Soft-Landing Trajectory and MPC Path"
filename = "RocketGroundThrustBound_BarelyFeasible3"

plt_x = plot(x_orig, z_orig, label = "Original Planned Trajectory")
plot!(xs, zs, label = "MPC Tracked Trajectory")
xlabel!("x")
ylabel!("z (height)")
title!(titleText)

In [ ]:
xmin = minimum(xs)
xmax = maximum(xs)

zmin = minimum(zs)
zmax = maximum(zs)

@gif for i in 1:size(xs, 1)
    plt = plot(xs[1:i], zs[1:i], label = "Trajectory", legend = :topleft)
    xlabel!("x")
    ylabel!("z (height)")
    xlims!(xmin,xmax)
    ylims!(zmin,zmax)
    title!(titleText)
end

In [ ]:
plt_ux = plot(uxs, label = "Ux Control")
hline!([-ulateral_max, ulateral_max], linecolor = :grey, linestyle = :dash,
                label = "Max Lateral Thrust")
# xlabel!("time (s)")
ylabel!("control (N)")
title!("Controls over Time")

plt_uz = plot(uzs, label = "Uz Control")
hline!([umax], linecolor = :grey, linestyle = :dash, label = "Max Thrust")
xlabel!("time (s)")
ylabel!("control (N)")

plt_u = plot(plt_ux, plt_uz, layout = (2, 1))

Below is a GIF of the trajectory. This will take several seconds to generate.

## Lastly, let's visualize the results in 3D with Meshcat

To do this, we need to provide a Mesh to <code>MeshCat</code>. This is due with <code>FileIO</code> and <code>MeshIO</code>.

In [ ]:
vis = Visualizer() # Creates a MeshCat Window
render(vis) # Renders the window below

In [ ]:
function TrajOptPlots._set_mesh!(vis, model::Rocket)
    obj = joinpath(@__DIR__, "SimpleRocket.obj")
    quad_scaling = 0.085
    robot_obj = FileIO.load(obj)
    robot_obj.vertices .*= quad_scaling
    mat = MeshPhongMaterial(color=colorant"lightgrey")
    setobject!(vis["geom"], robot_obj, mat)
end

TrajOptPlots.set_mesh!(vis, model)

In [ ]:
Base.position(::Rocket, x::StaticVector) = SA[x[1], x[2], x[3]]
RobotDynamics.orientation(::Rocket, x::StaticVector) = UnitQuaternion(0,0,0,1)

visualize!(vis, altro);

## Integrate the Dynamics to Verify the Dynamics Constraint

In Matlab, we would use ODE 45, but in the <code>DifferentialEquations.jl</code> package we can either (1) just let the <code>solve</code> method figure it out or (2) specify a solver such as <code>Tsit5</code>.

The controls change over time and are discretized. Below, a zero-order hold and a first-order hold are implemented for comparison. **We used a zero-order hold in the optimization so a zero-order hold should be used here.**

In [ ]:
get_controls_zero_order_hold(t) = U[min(Int(floor(t/dt)) + 1, size(U,1))]
f(y,p,t) = dynamics(model, y, get_controls_zero_order_hold(t))

states_y0 = x0
tspan = (0.0, tf)
controls_p0 = get_controls_zero_order_hold(tspan[1])

ode_prob_zoh = ODEProblem(f, states_y0, tspan, controls_p0)
@btime solve(ode_prob_zoh, Tsit5(), reltol=1e-10, abstol=1e-10,saveat=dt)
ode_sol_zoh = solve(ode_prob_zoh, Tsit5(), reltol=1e-10, abstol=1e-10,saveat=dt);

In [ ]:
function get_controls_first_order_hold(t)
    # dt and U are both global variables
    tn = t/dt
    
    t_below = Int(max(1, floor(tn)))
    t_above = Int(min(size(U, 1), ceil(tn)))
    
    if t_below >= t_above
        return U[t_below]
    end
    
    slope = (U[t_above] - U[t_below])
    return slope * (tn - t_below) + U[t_below]

end


f(y,p,t) = dynamics(model, y, get_controls_first_order_hold(t))

states_y0 = x0
tspan = (0.0, tf)
controls_p0 = get_controls_first_order_hold(tspan[1])

ode_prob_foh = ODEProblem(f, states_y0, tspan, controls_p0)
@btime solve(ode_prob_foh, Tsit5(), reltol=1e-10, abstol=1e-10,saveat=dt)
ode_sol_foh = solve(ode_prob_foh, Tsit5(), reltol=1e-10, abstol=1e-10,saveat=dt);

In [ ]:
err_zoh = norm.(ode_sol_zoh.u - X[1:(Int(floor(tspan[2]/dt))+1)])
err_foh = norm.(ode_sol_foh.u - X[1:(Int(floor(tspan[2]/dt))+1)])

plot(ode_sol_zoh.t, err_zoh, yaxis = :log, label = "Zero-Order Hold")
plot!(ode_sol_foh.t, err_foh, yaxis = :log, label = "First-Order Hold")
xlabel!("Time (s)")
ylabel!("Dynamics Error")
title!("Comparison in State Vector expected from ALTRO Solver \n to Integrated Dynamics Model")